In [1]:
import duckdb
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")
raw_con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/match_player_raw.duckdb")

In [2]:
con.execute("DROP TABLE IF EXISTS hero_synergy_trends")
con.execute("DROP TABLE IF EXISTS hero_counter_trends")

In [3]:
from services import database_functions as dbf

dbf.create_hero_counter_trends(con)
query = con.execute("SELECT * FROM hero_counter_trends LIMIT 1").fetchone()
print(query)
hero_counter_trends_col = con.execute("""
        PRAGMA table_info('hero_counter_trends');
        """).fetchall()
print(hero_counter_trends_col)

dbf.create_hero_synergy_trends(con)
query = con.execute("SELECT * FROM hero_synergy_trends LIMIT 1").fetchone()
print(query)
hero_counter_trends_col = con.execute("""
        PRAGMA table_info('hero_synergy_trends');
        """).fetchall()
print(hero_counter_trends_col)

None
[(0, 'assists', 'INTEGER', False, None, False), (1, 'creeps', 'INTEGER', False, None, False), (2, 'deaths', 'INTEGER', False, None, False), (3, 'denies', 'INTEGER', False, None, False), (4, 'enemy_assists', 'INTEGER', False, None, False), (5, 'enemy_creeps', 'INTEGER', False, None, False), (6, 'enemy_deaths', 'INTEGER', False, None, False), (7, 'enemy_denies', 'INTEGER', False, None, False), (8, 'enemy_hero_id', 'INTEGER', True, None, False), (9, 'enemy_kills', 'INTEGER', False, None, False), (10, 'enemy_last_hits', 'INTEGER', False, None, False), (11, 'enemy_networth', 'INTEGER', False, None, False), (12, 'enemy_obj_damage', 'INTEGER', False, None, False), (13, 'hero_id', 'INTEGER', True, None, False), (14, 'kills', 'INTEGER', False, None, False), (15, 'last_hits', 'INTEGER', False, None, False), (16, 'matches_played', 'INTEGER', False, None, False), (17, 'networth', 'INTEGER', False, None, False), (18, 'obj_damage', 'INTEGER', False, None, False), (19, 'wins', 'INTEGER', False, 

In [ ]:
query = con.execute(f"SELECT AVG(wins) FROM hero_synergy_trends").fetchdf()
print(query)

In [4]:
con.close()
raw_con.close()

Claude SQL to count matches in matches where all 12 players exist in player_matches_history with at least 5 matches per player.

In [ ]:
import duckdb
import pandas as pd

# Connect to the database
con = duckdb.connect("data/deadlock.db")

# Query to find matches where all 12 players have player trends 
# and at least 5 references in player_matches_history
query = """
WITH match_player_counts AS (
    -- Count players per match
    SELECT 
        m.match_id,
        COUNT(DISTINCT pm.account_id) AS player_count
    FROM 
        matches m
    JOIN 
        player_matches pm ON m.match_id = pm.match_id
    GROUP BY 
        m.match_id
),
players_with_trends AS (
    -- Find players who have trends
    SELECT 
        pm.match_id,
        pm.account_id,
        CASE WHEN pt.account_id IS NOT NULL THEN 1 ELSE 0 END AS has_trend
    FROM 
        player_matches pm
    LEFT JOIN 
        player_trends pt ON pm.account_id = pt.account_id
),
players_with_history AS (
    -- Count history entries per player
    SELECT 
        pm.match_id,
        pm.account_id,
        COUNT(pmh.match_id) AS history_count
    FROM 
        player_matches pm
    LEFT JOIN 
        player_matches_history pmh ON pm.account_id = pmh.account_id
    GROUP BY 
        pm.match_id, pm.account_id
),
match_trend_stats AS (
    -- Aggregate player trend stats by match
    SELECT 
        pwt.match_id,
        SUM(pwt.has_trend) AS players_with_trends,
        COUNT(*) AS total_players
    FROM 
        players_with_trends pwt
    GROUP BY 
        pwt.match_id
),
match_history_stats AS (
    -- Aggregate player history stats by match
    SELECT 
        pwh.match_id,
        COUNT(CASE WHEN pwh.history_count >= 5 THEN 1 END) AS players_with_5plus_history,
        COUNT(*) AS total_players  
    FROM 
        players_with_history pwh
    GROUP BY 
        pwh.match_id
)
-- Final results with matches meeting all criteria
SELECT 
    COUNT(*) AS matching_matches_count,
    (SELECT COUNT(*) FROM matches) AS total_matches_count
FROM 
    match_player_counts mpc
JOIN 
    match_trend_stats mts ON mpc.match_id = mts.match_id
JOIN 
    match_history_stats mhs ON mpc.match_id = mhs.match_id
WHERE 
    mpc.player_count = 12 -- Ensure match has exactly 12 players
    AND mts.players_with_trends = 12 -- All 12 players have trends
    AND mhs.players_with_5plus_history = 12; -- All 12 players have 5+ history entries
"""

# Execute the query and get results
result = con.execute(query).fetchdf()
print(f"Matches where all 12 players have trends and 5+ history entries: {result.iloc[0, 0]} out of {result.iloc[0, 1]} total matches")

# Get a sample of these matches (first 10)
sample_query = """
WITH match_player_counts AS (
    SELECT 
        m.match_id,
        COUNT(DISTINCT pm.account_id) AS player_count
    FROM 
        matches m
    JOIN 
        player_matches pm ON m.match_id = pm.match_id
    GROUP BY 
        m.match_id
),
players_with_trends AS (
    SELECT 
        pm.match_id,
        pm.account_id,
        CASE WHEN pt.account_id IS NOT NULL THEN 1 ELSE 0 END AS has_trend
    FROM 
        player_matches pm
    LEFT JOIN 
        player_trends pt ON pm.account_id = pt.account_id
),
players_with_history AS (
    SELECT 
        pm.match_id,
        pm.account_id,
        COUNT(pmh.match_id) AS history_count
    FROM 
        player_matches pm
    LEFT JOIN 
        player_matches_history pmh ON pm.account_id = pmh.account_id
    GROUP BY 
        pm.match_id, pm.account_id
),
match_trend_stats AS (
    SELECT 
        pwt.match_id,
        SUM(pwt.has_trend) AS players_with_trends,
        COUNT(*) AS total_players
    FROM 
        players_with_trends pwt
    GROUP BY 
        pwt.match_id
),
match_history_stats AS (
    SELECT 
        pwh.match_id,
        COUNT(CASE WHEN pwh.history_count >= 5 THEN 1 END) AS players_with_5plus_history,
        COUNT(*) AS total_players  
    FROM 
        players_with_history pwh
    GROUP BY 
        pwh.match_id
),
qualifying_matches AS (
    SELECT 
        mpc.match_id
    FROM 
        match_player_counts mpc
    JOIN 
        match_trend_stats mts ON mpc.match_id = mts.match_id
    JOIN 
        match_history_stats mhs ON mpc.match_id = mhs.match_id
    WHERE 
        mpc.player_count = 12
        AND mts.players_with_trends = 12
        AND mhs.players_with_5plus_history = 12
)
SELECT 
    m.match_id,
    m.start_time,
    m.game_mode,
    m.match_mode
FROM 
    matches m
JOIN 
    qualifying_matches qm ON m.match_id = qm.match_id
ORDER BY 
    m.start_time DESC
LIMIT 10;
"""

# Print sample of qualifying matches
sample_matches = con.execute(sample_query).fetchdf()
if not sample_matches.empty:
    print("\nSample of matches meeting criteria:")
    print(sample_matches)
else:
    print("\nNo matches found meeting all criteria")

# Close the connection
con.close()